In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['AR']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('androgen receptor', 8389),
 ('allergic rhinitis', 555),
 ('aldose reductase', 509),
 ('acrosome reaction', 325),
 ('amphiregulin', 180),
 ('acute rejection', 164),
 ('autosomal recessive', 109),
 ('aortic regurgitation', 98),
 ('androgen', 94),
 ('adrenoceptor', 80),
 ('adventitious root', 59),
 ('augmented reality', 48),
 ('adaptive response', 45),
 ('airway response', 26),
 ('aspirin resistant', 26),
 ('antigen retrieval', 24),
 ('astragali radix', 23),
 ('androgen receptor gene', 22),
 ('ankyrin repeat', 21),
 ('acid resistant', 20),
 ('antibiotic resistant', 19),
 ('ar', 18),
 ('aortic valve regurgitation', 15),
 ('acid rain', 14),
 ('antioxidant response', 14),
 ('autotrophic respiration', 13),
 ('after ripening', 13),
 ('adrenoreceptor', 13),
 ('allograft rejection', 13),
 ('autoregressive', 13),
 ('arginase', 12),
 ('antireflection', 11),
 ('ar a014418', 11),
 ('analysis ready', 10),
 ('anatomical resection', 10),
 ('anhydroretinol', 10),
 ('anoikis resistant', 10),
 ('activa

In [7]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [8]:
names

{'HGNC:644': 'AR',
 'DOID:DOID:4481': 'allergic rhinitis',
 'GO:GO:0007340': 'Acrosome Reaction',
 'HGNC:651': 'AREG',
 'HP:HP:0000007': 'Autosomal recessive inheritance',
 'MESH:D001022': 'Aortic Valve Insufficiency',
 'CHEBI:CHEBI:50113': 'androgen',
 'EFO:0000989': 'root structure',
 'MESH:D000081411': 'Augmented Reality',
 'MESH:D017089': 'Ankyrin Repeat',
 'MESH:D015258': 'Acid Rain',
 'MESH:D012119': 'Respiration',
 'MESH:C031469': 'anhydrovitamin A',
 'MESH:C470430': 'Amplex Red',
 'MESH:D006706': 'Homeostasis',
 'CHEBI:CHEBI:33288': 'rodenticide',
 'MESH:D012399': 'Rotation',
 'MESH:D012222': 'Rhinitis, Atrophic',
 'CHEBI:CHEBI:29952': 'L-arginine residue',
 'MESH:D011995': 'Recombination, Genetic',
 'CHEBI:CHEBI:63933': 'Aroclor 1254',
 'MESH:D005796': 'Genes',
 'MESH:D056151': 'Airway Remodeling',
 'MESH:D000707': 'Anaphylaxis',
 'MESH:C004468': 'Alizarin Red S',
 'MESH:C078098': 'ar-turmerone',
 'MESH:C574902': 'Amalakirasayana',
 'MESH:D012306': 'Risk',
 'CHEBI:CHEBI:40863'

In [9]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-11 21:14:55] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [10]:
result = [grounding_map, names, pos_labels]

In [11]:
result

[{'a ringens': 'ungrounded',
  'abstinent recovery': 'ungrounded',
  'accommodation response': 'ungrounded',
  'acid rain': 'ungrounded',
  'acid region': 'ungrounded',
  'acid resistant': 'ungrounded',
  'acoustic radiation': 'ungrounded',
  'acquired resistant': 'ungrounded',
  'acrosome reacted': 'ungrounded',
  'acrosome reaction': 'GO:GO:0007340',
  'activation rate': 'ungrounded',
  'activation ratio': 'ungrounded',
  'activation recovery': 'ungrounded',
  'activation reduction': 'ungrounded',
  'acute rejection': 'ungrounded',
  'acute restraint': 'ungrounded',
  'adaptive response': 'ungrounded',
  'adrenoceptor': 'FPLX:ADRB',
  'adrenodoxin reductase': 'ungrounded',
  'adrenoreceptor': 'ungrounded',
  'advanced reconstruction': 'ungrounded',
  'adventitious root': 'ungrounded',
  'adverse reaction': 'ungrounded',
  'affect recognition': 'ungrounded',
  'after ripening': 'ungrounded',
  'age refuse': 'ungrounded',
  'airway reactivation': 'ungrounded',
  'airway remodeling': 'M

In [14]:
grounding_map, names, pos_labels = [{'a ringens': 'ungrounded',
  'abstinent recovery': 'ungrounded',
  'accommodation response': 'ungrounded',
  'acid rain': 'ungrounded',
  'acid region': 'ungrounded',
  'acid resistant': 'ungrounded',
  'acoustic radiation': 'ungrounded',
  'acquired resistant': 'ungrounded',
  'acrosome reacted': 'ungrounded',
  'acrosome reaction': 'GO:GO:0007340',
  'activation rate': 'ungrounded',
  'activation ratio': 'ungrounded',
  'activation recovery': 'ungrounded',
  'activation reduction': 'ungrounded',
  'acute rejection': 'ungrounded',
  'acute restraint': 'ungrounded',
  'adaptive response': 'ungrounded',
  'adrenoceptor': 'FPLX:ADRB',
  'adrenodoxin reductase': 'ungrounded',
  'adrenoreceptor': 'ungrounded',
  'advanced reconstruction': 'ungrounded',
  'adventitious root': 'ungrounded',
  'adverse reaction': 'ungrounded',
  'affect recognition': 'ungrounded',
  'after ripening': 'ungrounded',
  'age refuse': 'ungrounded',
  'airway reactivation': 'ungrounded',
  'airway remodeling': 'MESH:D056151',
  'airway response': 'ungrounded',
  'aldose reductase': 'HGNC:381',
  'alizarin red': 'CHEBI:CHEBI:40863',
  'alizarin red s': 'MESH:C004468',
  'alkylresorcinols': 'ungrounded',
  'allelic ratio': 'ungrounded',
  'allelic replacement': 'ungrounded',
  'allergic rhinitis': 'NCIT:C79532',
  'allograft rejection': 'ungrounded',
  'amalaki rasayana': 'MESH:C574902',
  'ambiguity resolution': 'ungrounded',
  'amphiregulin': 'HGNC:651',
  'amplex red': 'MESH:C470430',
  'amplitude ratio': 'ungrounded',
  'analysis ready': 'ungrounded',
  'anaphylactoid reaction': 'MESH:D000707',
  'anatomical resection': 'ungrounded',
  'and resistant': 'ungrounded',
  'andreev reflection': 'ungrounded',
  'androgen': 'HGNC:644',
  'androgen receptor': 'HGNC:644',
  'androgen receptor gene': 'HGNC:644',
  'androgen response': 'ungrounded',
  'angle of resonance': 'ungrounded',
  'angle range': 'ungrounded',
  'anhydroretinol': 'MESH:C031469',
  'animals research': 'ungrounded',
  'ankyrin repeat': 'ungrounded',
  'anoikis resistant': 'ungrounded',
  'anoxia reoxygenation': 'ungrounded',
  'anthelmintic resistant': 'ungrounded',
  'antibiotic resistant': 'ungrounded',
  'anticipated regret': 'ungrounded',
  'anticoagulant rodenticides': 'CHEBI:CHEBI:33288',
  'antigen removal': 'ungrounded',
  'antigen retrieval': 'ungrounded',
  'antimicrobial resistant': 'ungrounded',
  'antioxidant response': 'ungrounded',
  'antireflection': 'ungrounded',
  'aortic regurgitation': 'MESH:D001022',
  'aortic root': 'ungrounded',
  'aortic valve regurgitation': 'MESH:D001022',
  'apparent retention': 'ungrounded',
  'appendiceal rupture': 'ungrounded',
  'applied relaxation': 'ungrounded',
  'ar': 'ungrounded',
  'ar a014418': 'ungrounded',
  'ar gene': 'ungrounded',
  'ar turmerone': 'MESH:C078098',
  'archesporial': 'ungrounded',
  'archesporial cells': 'ungrounded',
  'arg': 'CHEBI:CHEBI:29952',
  'arginase': 'ungrounded',
  'armored': 'ungrounded',
  'aroclor 1254': 'CHEBI:CHEBI:63933',
  'artificial rearing': 'ungrounded',
  'artificial reef': 'ungrounded',
  'arvanil': 'MESH:C121153',
  'asparagus racemosus': 'ungrounded',
  'aspect ratio': 'ungrounded',
  'aspirin resistant': 'ungrounded',
  'astragali radix': 'ungrounded',
  'asynchronous release': 'ungrounded',
  'at risk': 'ungrounded',
  'atp recognition': 'ungrounded',
  'atrophic rhinitis': 'MESH:D012222',
  'attack rate': 'ungrounded',
  'attributed risk': 'ungrounded',
  'auger recombination': 'MESH:D011995',
  'augmented reality': 'MESH:D000081411',
  'autoregressive': 'ungrounded',
  'autoregulation': 'MESH:D006706',
  'autoregulatory': 'ungrounded',
  'autosomal recessive': 'ungrounded',
  'autotrophic respiration': 'ungrounded',
  'autotrophic soil respiration': 'MESH:D012119',
  'average referred': 'ungrounded',
  'avoiding reaction': 'ungrounded',
  'axial rotation': 'ungrounded',
  'resorcylic acid hydrochloride dihydrate': 'ungrounded'},
 {'GO:GO:0007340': 'Acrosome Reaction',
  'FPLX:ADRB': 'ADRB',
  'MESH:D056151': 'Airway Remodeling',
  'HGNC:381': 'AKR1B1',
  'CHEBI:CHEBI:40863': '3,4-dihydroxy-9,10-dioxo-9,10-dihydroanthracene-2-sulfonic acid',
  'MESH:C004468': 'Alizarin Red S',
  'NCIT:C79532': 'Allergic Rhinitis',
  'MESH:C574902': 'Amalakirasayana',
  'HGNC:651': 'AREG',
  'MESH:C470430': 'Amplex Red',
  'MESH:D000707': 'Anaphylaxis',
  'HGNC:644': 'AR',
  'MESH:C031469': 'anhydrovitamin A',
  'CHEBI:CHEBI:33288': 'rodenticide',
  'MESH:D001022': 'Aortic Valve Insufficiency',
  'MESH:C078098': 'ar-turmerone',
  'CHEBI:CHEBI:29952': 'L-arginine residue',
  'CHEBI:CHEBI:63933': 'Aroclor 1254',
  'MESH:C121153': 'arvanil',
  'MESH:D012222': 'Rhinitis, Atrophic',
  'MESH:D011995': 'Recombination, Genetic',
  'MESH:D000081411': 'Augmented Reality',
  'MESH:D006706': 'Homeostasis',
  'MESH:D012119': 'Respiration'},
 ['FPLX:ADRB', 'HGNC:381', 'HGNC:644', 'HGNC:651']]

In [15]:
excluded_longforms = ['ar', 'ar gene']

In [16]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [17]:
additional_entities = {}

In [18]:
unambiguous_agent_texts = {}

In [19]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [20]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [21]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [22]:
intersection1

[]

In [23]:
intersection2

[]

In [24]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [25]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [26]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 21:21:14] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 21:26:11] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9814293262293461 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [27]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:63933': 1,
  'MESH:D012222': 7,
  'HGNC:651': 142,
  'GO:GO:0007340': 290,
  'HGNC:644': 6093,
  'ungrounded': 645,
  'HGNC:381': 382,
  'FPLX:ADRB': 65,
  'MESH:D006706': 5,
  'MESH:D001022': 87,
  'MESH:D000707': 4,
  'MESH:C031469': 9,
  'NCIT:C79532': 397,
  'MESH:D056151': 4,
  'MESH:C470430': 6,
  'CHEBI:CHEBI:40863': 4,
  'MESH:C078098': 1,
  'MESH:C121153': 1,
  'CHEBI:CHEBI:33288': 1,
  'MESH:D011995': 4,
  'MESH:D000081411': 30,
  'MESH:C574902': 3,
  'CHEBI:CHEBI:29952': 1,
  'MESH:D012119': 1,
  'MESH:C004468': 1},
 'f1': {'mean': 0.981429, 'std': 0.004942},
 'precision': {'mean': 0.978907, 'std': 0.005502},
 'recall': {'mean': 0.984436, 'std': 0.004371},
 'MESH:C470430': {'f1': {'mean': 0.36, 'std': 0.445421},
  'pr': {'mean': 0.4, 'std': 0.489898},
  'rc': {'mean': 0.333333, 'std': 0.421637}},
 'HGNC:381': {'f1': {'mean': 0.925903, 'std': 0.029163},
  'pr': {'mean': 0.900649, 'std': 0.037293},
  'rc': {'mean': 0.95333, 'std': 0.030564}

In [28]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [29]:
disamb.dump(model_name, results_path)

In [30]:
print(disamb.info())

Disambiguation model for AR

Produces the disambiguations:
	3,4-dihydroxy-9,10-dioxo-9,10-dihydroanthracene-2-sulfonic acid	CHEBI:CHEBI:40863
	ADRB*	FPLX:ADRB
	AKR1B1*	HGNC:381
	AR*	HGNC:644
	AREG*	HGNC:651
	Acrosome Reaction	GO:GO:0007340
	Airway Remodeling	MESH:D056151
	Alizarin Red S	MESH:C004468
	Allergic Rhinitis	NCIT:C79532
	Amalakirasayana	MESH:C574902
	Amplex Red	MESH:C470430
	Anaphylaxis	MESH:D000707
	Aortic Valve Insufficiency	MESH:D001022
	Aroclor 1254	CHEBI:CHEBI:63933
	Augmented Reality	MESH:D000081411
	Homeostasis	MESH:D006706
	L-arginine residue	CHEBI:CHEBI:29952
	Recombination, Genetic	MESH:D011995
	Respiration	MESH:D012119
	Rhinitis, Atrophic	MESH:D012222
	anhydrovitamin A	MESH:C031469
	ar-turmerone	MESH:C078098
	arvanil	MESH:C121153
	rodenticide	CHEBI:CHEBI:33288

Class level metrics:
--------------------
Grounding                                                      	Count	F1     
                                                             AR*	6093	0.98957
         

In [31]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1